In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import sys
from typing import List
import numpy as np
import joblib
import imodelsx.util
import sasc.viz
import pickle as pkl
from copy import deepcopy
from numpy.linalg import norm
from sasc.config import CACHE_DIR, RESULTS_DIR
import sasc.modules.fmri_module
cache_ngrams_dir = '/home/chansingh/mntv1/deep-fMRI/sasc/mprompt/cache/cache_ngrams'
regions_idxs_dir = '/home/chansingh/mntv1/deep-fMRI/sasc/brain_regions'

In [ ]:
ngrams_list = joblib.load(join(cache_ngrams_dir, 'fmri_UTS02_ngrams.pkl'))
rois_dict = joblib.load(join(regions_idxs_dir, 'rois_S02.jbl'))

In [ ]:
rois = ['RSC', 'OPA', 'PPA']
for k in rois:
    assert k in rois_dict

In [ ]:
embs = joblib.load(join(cache_ngrams_dir, 'fmri_embs.pkl'))

In [ ]:
mod = sasc.modules.fmri_module.fMRIModule(
    subject="UTS02",
    checkpoint="facebook/opt-30b",
    init_model=False,
    restrict_weights=False,
)
voxel_preds = mod(embs=embs, return_all=True)

In [ ]:
rois_dict[k]

In [ ]:
voxel_preds.shape

In [ ]:
outputs_dict = {
    k: voxel_preds[:, np.array(rois_dict[k])].mean(axis=1)
    for k in rois_dict
}
voxel_preds.shape

In [ ]:
# joblib.dump(outputs_dict, join(cache_ngrams_dir, 'rois_ngram_outputs_dict.pkl'))

In [51]:
df = pd.DataFrame(outputs_dict, index=ngrams_list[:len(voxel_preds)])[
    ['RSC', 'OPA', 'PPA']]
df.head()
top_ngrams_dict = {}
for k in df.columns:
    top_ngrams_dict[k] = df.sort_values(
        k, ascending=False).index[:100].tolist()
    df[k + '_only'] = df[k] - \
        df[[c for c in df.columns if c != k]].mean(axis=1)
    top_ngrams_dict[k + '_only'] = df.sort_values(
        k + '_only', ascending=False).index[:100].tolist()
top_ngrams_df = pd.DataFrame(top_ngrams_dict)
# display all rows
with pd.option_context('display.max_rows', None):
    display(top_ngrams_df)

,RSC,RSC_only,OPA,OPA_only,PPA,PPA_only
0,moved to vermont,came to florida,onto the railing,towards the ceiling,mile of cornfields,ranch with
1,in downriver michigan,traveled to marrakesh,towards the ceiling,onto the railing,beautiful moonlit mountains,adorable little bouquet
2,college in boston,moved to london,over the gulf,on the ceiling,giant stone cliffs,little bouquet
3,in middlebury vermont,college in boston,towards the river,against the railing,and shimmering skyscrapers,blueberry coffee cake
4,drove from vermont,moved to vermont,against the railing,on the railing,shining skyscrapers,like burnt steak
5,cabin in vermont,off in vancouver,onto the sidewalk,seats behind,on a dock,cheapest bouquet
6,northern midwest,home in minneapolis,on the railing,you sit backward,room with concrete,eggs and ham
7,in warmer mexico,triathlon in utah,that distant shore,feet hanging over,lakes and manicured,of torn fishnet
8,in nashville tennessee,year in ohio,towards the doors,towards the doors,contain strip malls,smallest cheapest bouquet
9,upstate new york,vancouver in canada,see the horizon,the door behind,shimmering skyscrapers,salad and stale


In [47]:
gpt4 = imodelsx.llm.get_llm('gpt-4-turbo-0125-spot')

explanations = {}
for k in top_ngrams_df.columns:

    s = '- ' + '\n- '.join(top_ngrams_df[k].iloc[:60])
    prompt = f'''Here is a list of phrases:
    {s}

    What is a common theme among these phrases? Return only a concise phrase.'''

    explanations[k] = gpt4(prompt)

cached!
cached!
cached!
cached!
cached!
cached!


In [48]:
explanations

{'RSC': 'Geographic locations',
 'RSC_only': 'Travel and locations',
 'OPA': 'Spatial descriptions',
 'OPA_only': 'Spatial orientation and direction',
 'PPA': 'Descriptive settings and landscapes',
 'PPA_only': 'Randomness and variety'}